In [4]:
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests as rq
from pathlib import Path
from converte import kp_deci, kp_in

In [10]:
dados = Path('dados')

In [3]:
# caminho para os dados
url = 'https://www.spaceweatherlive.com/en/auroral-activity/top-50-geomagnetic-storms.html'

# cabeçalho que simula a requisição do navegador


cookies = {
    'CookieScriptConsent': '{"action":"accept","categories":"[\\"performance\\",\\"targeting\\",\\"functionality\\"]","CMP":"CPWNoYVPWNoYVF2ADBENBdCgAAAAAAAAAAAAAAAAAAAA.YAAAAAAAAAAA","key":"8aacd5d7-1b3e-46ab-a31d-3053f9c55c07"}',
    'SPSI': '575682a63cef91001aa58345e475e5e5',
    'SPSE': 'K8vuNQIvLP/gqu8okpRWkrqnBlfqrNBLNY1uCCXs3SV+TndFT9w0hKadmkQkjQeBym/SGDPFGLNlgL2H7AqMWg==',
    'spcsrf': 'ea702eee185ffd267e93cdb0d12c43c9',
    'PRLST': 'cb',
    'UTGv2': 'h4684ba6fe42242c1a648063d633336c2760',
    '_ga_25HHLX0QL3': 'GS1.1.1650241884.3.0.1650241884.0',
    '_ga': 'GA1.2.1277228958.1647879971',
    '_gid': 'GA1.2.1948256143.1650241896',
    'adOtr': '8U7O5653aec',
}

headers = {
    'authority': 'www.spaceweatherlive.com',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'accept-language': 'pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7',
    'cache-control': 'max-age=0',
    # Requests sorts cookies= alphabetically
    # 'cookie': 'CookieScriptConsent={"action":"accept","categories":"[\\"performance\\",\\"targeting\\",\\"functionality\\"]","CMP":"CPWNoYVPWNoYVF2ADBENBdCgAAAAAAAAAAAAAAAAAAAA.YAAAAAAAAAAA","key":"8aacd5d7-1b3e-46ab-a31d-3053f9c55c07"}; SPSI=575682a63cef91001aa58345e475e5e5; SPSE=K8vuNQIvLP/gqu8okpRWkrqnBlfqrNBLNY1uCCXs3SV+TndFT9w0hKadmkQkjQeBym/SGDPFGLNlgL2H7AqMWg==; spcsrf=ea702eee185ffd267e93cdb0d12c43c9; PRLST=cb; UTGv2=h4684ba6fe42242c1a648063d633336c2760; _ga_25HHLX0QL3=GS1.1.1650241884.3.0.1650241884.0; _ga=GA1.2.1277228958.1647879971; _gid=GA1.2.1948256143.1650241896; adOtr=8U7O5653aec',
    'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="100"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Linux"',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'none',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.60 Safari/537.36',
}

In [13]:
response = rq.get('https://www.spaceweatherlive.com/en/auroral-activity/top-50-geomagnetic-storms.html', headers=headers, cookies=cookies)

#valores para converter

# resposta da requisição
response = rq.get(url, headers=headers)

with open(f'{dados}/dadoshtml.txt', 'w') as file:
    file.writelines(str(response.content))


with open(f'{dados}/dadoshtml.txt', 'r') as file:
    sopa = bs(file, 'html.parser')


# separando a tabela do restante da pagina
tabela = sopa.find_all('table', attrs={'class':'table'})
tabela = tabela[0].find_all('tr')

# variavel que irá guardar minha lista de valores
g =[]

# colunas a serem utilizadas pra nomear as tabelas
colunas = ['data', 'Ap', 'kp_0h', 'kp_3h', 'kp_6h', 'kp_9h', 'kp_12h', 'kp_15h', 'kp_18h', 'kp_21h', 'Kp_Max_dia']

# interando sobre as linhas da tabela
for i in tabela[1:]:
    f = []
    c = i.find_all('td')
    for e in c[1:-1]:
        f.append(e.text)
    g.append(f)
# criando um datafra pandas
df = pd.DataFrame(g, columns=colunas)
# salvando a tabela dos valores do kp
df.to_string(f'{dados}/tempestades_kp.txt')

# convertendo os valores kp em kp decimais
for ki, kd in zip(kp_in, kp_deci):
    df.replace(ki, kd, inplace=True)

# salvando a tabela dos valores do kp  decimais
df.to_string(f'{dados}/tempestades_kp_decimais.txt')